# Exploratory Data Analysis (EDA) and Feature Engineering

In this notebook, I will perform exploratory data analysis (EDA) on the cleaned credit card customer data to better understand patterns, relationships, and distributions of variables that may influence customer churn.  

Insights from EDA will guide the creation of meaningful features that could improve the performance of our predictive models.  

### Note on Univariate Analysis

Univariate analysis for both numerical and categorical features was already performed during the **Data Cleaning** stage:

- **Numerical features:**  
  - **Skewness:** Specifically examined `Income`, `CreditLimit`, and `TotalSpend`, as these columns contained missing values and required careful handling for imputation and potential transformation.  
  - **Outliers:** All numerical features were checked for outliers using the **IQR method**. Outliers were **capped** rather than removed to preserve dataset size while reducing the influence of extreme values.

- **Categorical features:**  
  Frequency counts and distributions were assessed.  
  - Low-cardinality features were **one-hot encoded**.  
  - High-cardinality features were **frequency encoded** to retain predictive information without inflating dimensionality.

As a result, the dataset has already been cleaned and transformed at a univariate level, allowing this notebook to focus on:

- Bivariate analysis of features and the target variable  
- Correlation analysis  
- Feature Engineering
- Encoding categorical variables and preparing data for modeling

In [ ]:
# Import necessary libraries
import pandas as pd

# Load the cleaned dataset saved from the previous step
cleaned_data_path = r'..\..\data\processed\credit_card_attrition_cleaned.csv'
df = pd.read_csv(cleaned_data_path)

# Display the first few rows to verify loading
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

## 1. Bivariate analysis of features and the target variable
Goal: Identify features that differ significantly between churned (AttritionFlag=1) and non-churned (AttritionFlag=0) customers.

In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Count the number of churned and non-churned customers
attrition_counts = df['AttritionFlag'].value_counts()

# Pie chart
plt.figure(figsize=(6,6))
plt.pie(attrition_counts, labels=attrition_counts.index, autopct='%1.1f%%', startangle=90, colors=['#4CAF50', '#F44336'])
plt.title('Distribution of AttritionFlag')
plt.show()

In [ ]:
# Separate features by type
continuous_features = ['Age', 'Income', 'CreditLimit', 'TotalSpend', 'Tenure', 'TotalTransactions']
boolean_features = ['Is_Female', 'MaritalStatus_Divorced', 'MaritalStatus_Married', 
                    'MaritalStatus_Single', 'MaritalStatus_Widowed',
                    'EducationLevel_Bachelor', 'EducationLevel_High School',
                    'EducationLevel_Master', 'EducationLevel_PhD',
                    'CardType_Black', 'CardType_Gold', 'CardType_Platinum', 'CardType_Silver']
engineered_features = [f'Feature_{i}' for i in range(50)] + ['Country_FE']

In [ ]:
# -------------------------------
# 1. Continuous Features
# -------------------------------

print("Bivariate Analysis: Continuous Features\n")

for col in continuous_features:
    print(f"{col} vs AttritionFlag")
    print(df.groupby('AttritionFlag')[col].describe(), "\n")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Set folder to save plots
save_folder = r'..\..\reports\figures\EDA_FeatureEng\Continuous_FeaturesVSAttritionFlag'
os.makedirs(save_folder, exist_ok=True)

# Number of rows and columns
rows, cols = 2, 3

fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 4), sharex=True, sharey=False)
axes = axes.flatten()

for i, col in enumerate(continuous_features):
    sns.boxplot(ax=axes[i], x='AttritionFlag', y=col, data=df)
    axes[i].set_title(col, fontsize=10)
    axes[i].set_xlabel("")  # remove individual x-labels
    axes[i].set_ylabel("")  # remove individual y-labels

# Shared axis labels
fig.text(0.5, 0.04, 'AttritionFlag', ha='center', fontsize=12)
fig.text(0.04, 0.5, 'Value', va='center', rotation='vertical', fontsize=12)

plt.tight_layout(rect=[0.05, 0.05, 1, 1])

# Save single PNG
plt.savefig(os.path.join(save_folder, 'continuous_features_boxplots.png'))
plt.show()

*For Continuous features `Age`, `Income`, `CreditLimit`, `TotalSpend`, `Tenure`, `TotalTransactions`,  there are no significant differences in the distributions of continuous features between customers who churned (AttritionFlag = 1) and those who did not (AttritionFlag = 0). This suggests that these continuous features may have limited predictive power for distinguishing churn in this dataset.*

In [ ]:
# -------------------------------
# Boolean Features - Print Crosstabs
# -------------------------------
print("Bivariate Analysis: Boolean Features\n")

for col in boolean_features:
    print(f'{col} vs AttritionFlag')
    print(pd.crosstab(df[col], df['AttritionFlag'], normalize='columns'), "\n")  # proportions

In [ ]:
import math
import matplotlib.pyplot as plt
import seaborn as sns

save_folder = r'..\..\reports\figures\EDA_FeatureEng\Boolean_FeaturesVSAttritionFlag'
os.makedirs(save_folder, exist_ok=True)

n_features = len(boolean_features)
n_cols = 3
n_rows = math.ceil(n_features / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4), sharey=True, sharex=True)
axes = axes.flatten()  # Easier indexing

for i, col in enumerate(boolean_features):
    sns.countplot(x=col, hue='AttritionFlag', data=df, palette='Set2', ax=axes[i])
    axes[i].set_title(f'{col} by AttritionFlag')
    axes[i].set_xlabel("")  # Remove X-axis label
    axes[i].set_ylabel('Count')
    axes[i].legend(title='AttritionFlag', labels=['No Churn (0)', 'Churn (1)'])

# Remove empty subplots but keep last one centered if applicable
if n_features % n_cols != 0:
    empty_plots = n_cols - (n_features % n_cols)
    for j in range(1, empty_plots + 1):
        fig.delaxes(axes[-j])  # Remove unused axes

plt.tight_layout()
plt.savefig(os.path.join(save_folder, 'boolean_features_barplots.png'))
plt.show()

*Same with continuous variables, the boolean features do not show significant differences between Churn and No Churn.*

In [ ]:
# -------------------------------
# Engineered Features - Print Statistics
# -------------------------------
print("Bivariate Analysis: Engineered Features\n")

for col in engineered_features:
    print(f"{col} vs AttritionFlag")
    print(df.groupby('AttritionFlag')[col].mean(), "\n")

In [ ]:
import math
import matplotlib.pyplot as plt
import seaborn as sns
import os

save_folder = r'..\..\reports\figures\EDA_FeatureEng\Engineered_FeaturesVSAttritionFlag'
os.makedirs(save_folder, exist_ok=True)

n_features = len(engineered_features)
n_cols = 3
n_rows = math.ceil(n_features / n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4), sharey=False, sharex=True)
axes = axes.flatten()

for i, col in enumerate(engineered_features):
    sns.boxplot(x='AttritionFlag', y=col, data=df, ax=axes[i])
    axes[i].set_title(f'{col} vs AttritionFlag')
    axes[i].set_xlabel('')  # Remove X label
    axes[i].set_ylabel(col)

# Remove extra empty subplots
if n_features % n_cols != 0:
    empty_plots = n_cols - (n_features % n_cols)
    for j in range(1, empty_plots + 1):
        fig.delaxes(axes[-j])

plt.tight_layout()
plt.savefig(os.path.join(save_folder, 'engineered_features_boxplots.png'))
plt.show()

*The bivariate analysis of the engineered features (`Feature_0` to `eature_49` and `Country_FE`) against the target variable AttritionFlag shows that the mean values of each feature are very similar between employees who stayed (`0`) and those who left (`1`).*

### Bivariate Analysis Conclusion
The bivariate analysis shows minimal differences between the features and the target variable (`AttritionFlag`). Continuous, boolean, and engineered features all exhibit similar distributions across the target classes. Overall, no individual feature demonstrates a significant relationship with attrition, indicating that predictive patterns may require modeling feature interactions or more advanced techniques.

## 2. Correlation Analysis

Goal: Discover highly correlated features to inform which features to combine, transform, or drop during feature engineering.

In [ ]:
# Spearman correlation
corr_matrix = df.corr(method='spearman')

pd.set_option('display.max_rows', None) 

# Correlation with target
target_corr = corr_matrix['AttritionFlag'].sort_values(ascending=False)
print(target_corr)

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(x=target_corr.index, y=target_corr.values)
plt.xticks(rotation=90)
plt.ylabel('Spearman Correlation with AttritionFlag')
plt.title('Feature Correlation with Target')
plt.tight_layout()
plt.show()

*Spearman correlation between all numeric features and `AttritionFlag` shows values extremely close to zero, indicating no significant linear or monotonic relationship. This aligns with the bivariate analysis findings, confirming that none of the features individually differentiate the target.*

In [ ]:
plt.figure(figsize=(20,16))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, linewidths=0.5)
plt.title('Spearman Correlation Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

### Correlation Analysis Conclusion
*There are no notable correlations between feature-feature and feature-target relationships. The few negative correlations observed are expected, as they originate from individual columns that were one-hot encoded.*

## 3. Feature Engineering

Goal: To transform the existing dataset into a richer, more informative set of features that maximizes the model’s ability to predict attrition/churn, despite weak raw correlations.

In [ ]:
df.head()

### Create target-independent features

In [ ]:
import numpy as np

df['AvgTransaction'] = df['TotalSpend'] / np.where(df['TotalTransactions'] == 0, 1, df['TotalTransactions'])
df['CreditUsage'] = df['TotalSpend'] / np.where(df['CreditLimit'] == 0, 1, df['CreditLimit'])
df['SpendIncomeRatio'] = df['TotalSpend'] / np.where(df['Income'] == 0, 1, df['Income'])
df['TenureRatio'] = df['Tenure'] / np.where(df['Age'] == 0, 1, df['Age'])

`AvgTransaction`: Measures average spend per transaction, it normalizes spending by number of transactions to capture customer behavior.

`CreditUsage`: Indicates how much of their credit limit the customer uses; can reflect financial stress or engagement with the product.

`SpendIncomeRatio`: Shows the relative spending compared to income, which can indicate whether spending is sustainable or risky.

`TenureRatio`: Normalizes tenure by age, capturing how long a customer has been with the bank relative to their age (loyalty vs. age).

In [ ]:
feature_cols = [f'Feature_{i}' for i in range(50)]

df['Feature_sum'] = df[feature_cols].sum(axis=1)
df['Feature_mean'] = df[feature_cols].mean(axis=1)
df['Feature_std'] = df[feature_cols].std(axis=1)
df['Feature_max'] = df[feature_cols].max(axis=1)
df['Feature_min'] = df[feature_cols].min(axis=1)

Aggregates condense high-dimensional features into summary statistics:

sum: total activity/score across all features

mean: average behavior across features

std: variability of behavior

max/min: capture extremes or outliers

Helps reduce dimensionality and captures overall trends in customer behavior.

In [ ]:
#Checking for skewness

num_cols = df.select_dtypes(include=np.number).columns.tolist()
num_cols.remove('AttritionFlag')  # exclude target

skew_values = df[num_cols].skew().sort_values(ascending=False)
print(skew_values.head(20))  # top 20 most skewed features

In [ ]:
for col in ['CreditUsage', 'SpendIncomeRatio']:  # only skewed features
    min_val = df[col].min()
    if min_val <= 0:
        df[col + '_log'] = np.log1p(df[col] - min_val + 1)
    else:
        df[col + '_log'] = np.log1p(df[col])

*Since `CreditUsage` and `SpendIncomeRatio` are highly-skewed, we perform a log transformation (with a shift if necessary) to reduce skewness and make the distributions more suitable for modeling.*

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.head()

*Checking if there are correlations between features with newly added ones.*

In [ ]:
# Only numeric columns
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Compute correlation with target
corr_with_target = df[numeric_cols].corr()['AttritionFlag'].sort_values(ascending=False)

# Show top 10 positively and negatively correlated features
print("Top positive correlations:\n", corr_with_target.head(10))
print("\nTop negative correlations:\n", corr_with_target.tail(10))


In [ ]:
corr_spearman = df.corr(method='spearman')['AttritionFlag'].sort_values(ascending=False)
print("Top positive correlations:\n", corr_spearman.head(10))
print("\nTop negative correlations:\n", corr_spearman.tail(10))

In [ ]:
# Spearman correlation
corr_matrix = df.corr(method='spearman')

plt.figure(figsize=(20,16))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, linewidths=0.5)
plt.title('Spearman Correlation Matrix')
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.show()

All numeric features, including engineered ones, show very low correlations with the target (highest ~0.009, lowest ~-0.005).

Even features with low correlation are kept, as they may still contribute in combination with others or capture non-linear relationships.

This suggests that churn is likely driven by complex interactions, so feature engineering and non-linear models may be important.

### Create target-dependent features

### *Split dataset first to avoid data leakage since this will involve the target variable `AttritionFlag`*

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['AttritionFlag'])
y = df['AttritionFlag']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
train_df = X_train.join(y_train)

# Example: churn rate per card type
card_cols = ['CardType_Black', 'CardType_Gold', 'CardType_Platinum', 'CardType_Silver']
for col in card_cols:
    churn_rate = train_df.groupby(col)['AttritionFlag'].mean()
    X_train[col + '_ChurnRate'] = X_train[col].map(churn_rate)
    X_test[col + '_ChurnRate'] = X_test[col].map(churn_rate)

In [ ]:
numeric_feats = ['TotalSpend', 'CreditUsage', 'AvgTransaction']
mean_churned = train_df[train_df['AttritionFlag']==1][numeric_feats].mean()

for feat in numeric_feats:
    X_train[feat+'_diff_churn'] = X_train[feat] - mean_churned[feat]
    X_test[feat+'_diff_churn'] = X_test[feat] - mean_churned[feat]


In [ ]:
# Combine X_train and y_train for saving
train_save = X_train.copy()
train_save['AttritionFlag'] = y_train
train_save.to_csv(r'..\..\data\processed\train.csv', index=False)

# Combine X_test and y_test for saving
test_save = X_test.copy()
test_save['AttritionFlag'] = y_test
test_save.to_csv(r'..\..\data\processed\test.csv', index=False)

In [ ]:
train_save.head()

In [ ]:
test_save.head()

*After this code block, transfer to 3_ModelTraining.ipynb*

## 4. Handling Class Imbalance

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Check distribution
class_counts = df['AttritionFlag'].value_counts()
print(class_counts)

# Visualize
plt.figure(figsize=(5,4))
sns.barplot(x=class_counts.index, y=class_counts.values)
plt.title("AttritionFlag Distribution")
plt.xlabel("AttritionFlag")
plt.ylabel("Count")
plt.show()

*Since the target variable `AttritionFlag` is highly imbalance, we are going to apply SMOTE on the train data.*

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

In [ ]:
# Check target distribution after SMOTE
print("Training after SMOTE:\n", y_train_res.value_counts())
print("Test distribution:\n", y_test.value_counts())

# Remove scaling for XGBoost
# (Tree models don't require standardization)

## 5. Model Training

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Initialize model
xgb_model = XGBClassifier(
    n_estimators=300,         # number of boosting rounds
    learning_rate=0.1,        # step size shrinkage
    max_depth=5,              # maximum tree depth
    subsample=0.8,            # subsample ratio of training data
    colsample_bytree=0.8,     # subsample ratio of features
    random_state=42,
    eval_metric='logloss',
    n_jobs=-1
)

# Train on SMOTE-resampled data
xgb_model.fit(X_train, y_train)

# Predictions
y_pred = xgb_model.predict(X_test)
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Evaluation
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nROC-AUC Score:", roc_auc_score(y_test, y_pred_proba))

In [ ]:
import numpy as np

# Check first 20 predicted probabilities
print(y_pred_proba[:20])

# Compare probability distribution for each class in test set
print("\nMean probability for non-churners (class 0):", np.mean(y_pred_proba[y_test == 0]))
print("Mean probability for churners (class 1):", np.mean(y_pred_proba[y_test == 1]))

In [ ]:
print(y_train_res.value_counts())
print(y_test.value_counts())


In [ ]:
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]
print("Min prob:", y_pred_proba.min(), "Max prob:", y_pred_proba.max())
print("Mean prob for churners:", y_pred_proba[y_test == 1].mean())
print("Mean prob for non-churners:", y_pred_proba[y_test == 0].mean())

In [ ]:
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X_train_res, y_train_res)
print(classification_report(y_test, dummy.predict(X_test)))

In [ ]:
import pandas as pd

importances = pd.Series(xgb_model.feature_importances_, index=X_train_res.columns)
print(importances.sort_values(ascending=False).head(10))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Quick churn vs non-churn mean comparison
churn_means = X_train.groupby(y_train).mean()
print(churn_means.T.sort_values(by=1, ascending=False).head(15))

# Quick separation plot for a key numeric feature
feature = "CreditLimit"  # change as needed
plt.hist(X_train.loc[y_train == 0, feature], bins=30, alpha=0.5, label="Non-Churn")
plt.hist(X_train.loc[y_train == 1, feature], bins=30, alpha=0.5, label="Churn")
plt.legend()
plt.title(f"Distribution of {feature}")
plt.show()